# Swift SkyMap

What follows is a prototype to sum Swift-XRT images.
*Photon Counting* mode images will be used since they are simpler to deal with.

The pipeline start from a table of Swift observations, at file `SwiftXrt_master.csv`.
This file does not provide all the pointings, but a sub-sample considering the time exposure during `PC` mode.
Although not relevant, the table was extracted from ASDC's Swift database on July 7, 2017.

The pipeline goes like:

* select overlapping observations
* download observation files
* sum event-images
* sum exposure-maps

## Read data

First, get a glance of the file content/format (first rows, since it is an ascii/csv file).

Then, use Pandas to load the table.

In [1]:
!head -n3 SwiftXrt_master.csv

head: cannot open ‘SwiftXrt_master.csv’ for reading: No such file or directory


In [2]:
import pandas
pandas.options.display.max_colwidth = 300

In [4]:
table_master = pandas.read_csv('../SwiftXrt_master.csv', sep=';', header=0)
table_master

,NAME,ORIG_TARGET_I,TARGET_ID,RA,DEC,START_TIME,STOP_TIME,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC,ARCHIVE_DATE,SOFTWARE_VERS,PROCESSING_DA,PROCESSING_VE
0,NON-SCIENCE DATA,60142,60142,0.640524,0.278400,04/12/08 05:50:01,04/12/08 14:29:44,0,0,60142000,60142000,6665.174,2201.420,04/12/15 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/12 00:00:14,3.7.6
1,NON-SCIENCE DATA,60140,60140,0.640524,0.278400,04/12/07 23:59:16,04/12/08 06:37:25,0,0,60140000,60140000,10642.695,8751.410,04/12/14 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/14 00:00:14,3.7.6
2,NON-SCIENCE DATA,60150,60150,0.640524,0.278400,04/12/09 05:48:31,04/12/09 13:22:42,0,0,60150000,60150000,7307.151,2931.921,04/12/16 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/14 00:00:14,3.7.6
3,NON-SCIENCE DATA,60148,60148,0.640524,0.278400,04/12/08 23:59:14,04/12/09 06:44:02,0,0,60148000,60148000,12749.664,9399.110,04/12/15 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/14 00:00:14,3.7.6
4,NON-SCIENCE DATA,60156,60156,0.640524,0.278400,04/12/09 23:59:14,04/12/10 06:43:19,0,0,60156000,60156000,12687.397,9569.049,04/12/16 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/12 00:00:14,3.7.6
5,NON-SCIENCE DATA,60146,60146,0.640524,0.278400,04/12/08 17:17:18,04/12/09 00:39:19,0,0,60146000,60146000,3892.175,2716.944,04/12/15 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/13 00:00:14,3.7.6
6,NON-SCIENCE DATA,60138,60138,0.640524,0.278400,04/12/07 17:59:17,04/12/08 00:45:14,0,0,60138000,60138000,4074.440,2704.442,04/12/14 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/12 00:00:14,3.7.6
7,NON-SCIENCE DATA,60152,60152,0.640524,0.278400,04/12/09 11:42:05,04/12/10 00:05:14,0,0,60152000,60152000,6332.973,1080.260,04/12/16 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/13 00:00:14,3.7.6
8,4PBCJ2239.3-8920,85333,85333,339.655569,-89.385365,15/04/10 17:51:13,15/04/10 18:43:18,4,4,85333004,85333004,287.896,287.896,15/04/21 00:00:14,HEA_21DEC2012_V6.13_SWIFT_REL4.0(BLD29)_14DEC2012_,15/04/20 00:00:14,3.16.08
9,4PBCJ2239.3-8920,85333,85333,338.197063,-89.381632,14/12/30 20:24:13,15/01/01 22:34:07,2,2,85333002,85333002,870.060,870.019,15/01/11 00:00:14,HEA_21DEC2012_V6.13_SWIFT_REL4.0(BLD29)_14DEC2012_,15/01/10 00:00:14,3.16.07


## Select table entries

*Criteria* to select the observation from such table should be further explored, sure.
But for this prototype we may simply chose an object, and filter all observations accordingly.

In [53]:
# object_select = 'GRB110223A'
#object_select = 'BZBJ1134-1729'
# object_select = '1ES1218+304'
object_select ='GRB090813'
table_object = table_master.loc[table_master.NAME==object_select]

In [54]:
table_object

,NAME,ORIG_TARGET_I,TARGET_ID,RA,DEC,START_TIME,STOP_TIME,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC,ARCHIVE_DATE,SOFTWARE_VERS,PROCESSING_DA,PROCESSING_VE
104362,GRB090813,359884,359884,227.063646,88.565102,09/08/20 01:25:14,09/08/20 23:11:08,10,10,359884010,359884010,8558.704,8558.540,09/08/31 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03
104363,GRB090813,359884,359884,226.886510,88.573381,09/08/19 01:21:14,09/08/20 00:42:35,9,9,359884009,359884009,9857.969,9857.846,09/08/30 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03
104364,GRB090813,359884,359884,226.797554,88.574657,09/08/21 12:43:14,09/08/21 23:13:38,11,11,359884011,359884011,5525.620,5525.620,09/09/01 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/09 00:00:14,3.17.03
104365,GRB090813,359884,359884,226.831419,88.576467,09/08/18 01:17:14,09/08/19 00:39:07,8,8,359884008,359884008,8616.398,8616.176,09/08/29 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03
104367,GRB090813,359884,359884,226.619024,88.587938,09/08/17 01:13:14,09/08/17 19:46:13,7,7,359884007,359884007,8369.063,8368.879,09/08/28 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03
104368,GRB090813,359884,359884,224.868678,88.582777,09/08/13 04:10:14,09/08/14 00:18:06,3,3,359884003,359884003,6528.982,6528.974,09/08/24 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03
104370,GRB090813,359884,359884,226.667761,88.596243,09/08/16 01:09:14,09/08/17 00:31:18,6,6,359884006,359884006,5380.663,5380.545,09/08/27 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03
104371,GRB090813,359884,359884,224.551432,88.590625,09/08/14 01:01:14,09/08/15 00:27:16,4,4,359884004,359884004,6907.577,6907.572,09/08/25 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03
104372,GRB090813,359884,359884,225.694009,88.605731,09/08/12 03:55:03,09/08/12 22:25:20,0,0,359884000,359884000,27058.183,26207.813,09/08/23 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03
104374,GRB090813,359884,359884,226.811975,88.627077,09/08/15 01:06:43,09/08/16 00:28:57,5,5,359884005,359884005,3858.042,3857.993,09/08/26 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/04/08 00:00:14,3.17.03


### Check selection

In [55]:
table_object.describe()

,ORIG_TARGET_I,TARGET_ID,RA,DEC,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC
count,10.0,10.0,10.000000,10.000000,10.00000,10.00000,1.000000e+01,1.000000e+01,10.0000,10.000000
mean,359884.0,359884.0,226.279201,88.588000,6.30000,6.30000,3.598840e+08,3.598840e+08,9066.1201,8980.995800
std,0.0,0.0,0.908315,0.018235,3.40098,3.40098,3.400980e+00,3.400980e+00,6579.6416,6321.725057
min,359884.0,359884.0,224.551432,88.565102,0.00000,0.00000,3.598840e+08,3.598840e+08,3858.0420,3857.993000
25%,359884.0,359884.0,225.925263,88.575110,4.25000,4.25000,3.598840e+08,3.598840e+08,5776.4605,5776.458500
50%,359884.0,359884.0,226.732657,88.585358,6.50000,6.50000,3.598840e+08,3.598840e+08,7638.3200,7638.225500
75%,359884.0,359884.0,226.826558,88.594839,8.75000,8.75000,3.598840e+08,3.598840e+08,8601.9745,8601.767000
max,359884.0,359884.0,227.063646,88.627077,11.00000,11.00000,3.598840e+08,3.598840e+08,27058.1830,26207.813000


There are 5 selected entries, should be enough for this test.
We now have to download the respective files.

The Swift archive is organized by observation's start date (column `START_TIME`) and id (`OBSID`):
`ftp://legacy.gsfc.nasa.gov/swift/data/obs/`**START_TIME**`/`**OBSID**`/`.

In [39]:
def swift_archive_obs_path(date,obsid):
    '''
    Format `date` and `obsid` information for swift archive
    '''
    def extract_date(archive_date):
        '''
        Extract year/month from swift date format

        Example:
        "13/06/26 23:59:12" --> "2013_06"
        '''
        from datetime import datetime
        try:
            dt = datetime.strptime(archive_date,'%y/%m/%d %H:%M:%S')
        except:
            print(archive_date)
            return None
        year_month = '{:4d}_{:02d}'.format(dt.year,dt.month)
        return year_month

    dtf = extract_date(date)
    if dtf is None:
        return None
    obs = '{:011d}'.format(obsid)
    return '{}/{}'.format(dtf,obs)



fileout='1ES_object_observations.csv'
table_object.apply(lambda x:swift_archive_obs_path(x.START_TIME,x.OBSID), axis=1).to_csv(fileout, index=False)
!cat $fileout

2017_02/00030376079
2016_03/00030376060
2017_02/00030376080
2016_02/00030376059
2017_04/00030376087
2016_05/00030376072
2016_03/00030376062
2017_04/00030376086
2016_03/00030376063
2016_05/00030376073
2017_01/00030376076
2016_04/00030376067
2016_04/00030376071
2014_03/00030376033
2014_02/00030376032
2017_03/00030376083
2016_04/00030376069
2016_03/00030376061
2017_05/00030376089
2016_04/00030376068
2006_05/00030376006
2006_05/00030376005
2016_04/00030376070
2017_03/00030376084
2016_02/00030376057
2016_02/00030376058
2017_04/00030376085
2017_04/00030376088
2017_01/00030376075
2016_02/00030376056
2017_01/00030376078
2017_01/00030376077
2016_12/00030376074
2006_05/00030376004
2006_05/00030376003
2006_03/00030376001
2006_03/00030376002


## Download observation files

To download the files, let's use a script in the background (the script is written in Bash and calls `wget`).

$ `./download_obslist.sh <fileout>`

After a couple of minutes, we should have the files like shown below.
(For some reason, the "date" part of observation `00085333007` is associated with its `ARCHIVE_DATE`, i.e, `2015_05` and was not downloaded now, which should not be a problem for us, anyway.)

In [41]:
!ls swift/data/obs/*

swift/data/obs/2011_02:
00446674000  00446674003  00446674006  00446674008
00446674002  00446674004  00446674007

swift/data/obs/2011_03:
00446674011  00446674012  00446674019

swift/data/obs/2012_08:
00091401002  00091401003

swift/data/obs/2012_11:
00091401005  00091401006  00091401007  00091401008  00091401009

swift/data/obs/2013_01:
00091401011  00091401012

swift/data/obs/2013_03:
00091401015

swift/data/obs/2014_11:
00085333001

swift/data/obs/2014_12:
00085333002

swift/data/obs/2015_04:
00085333004

swift/data/obs/2015_07:
00085333009


## Create the image stacks

There is also a shell script for doing this task(s) in the background.

The shell command line goes like:
```
source setup_ximage_files.fsh 

EVENTSFILE='object_events.txt'
EXMAPSFILE='object_exmaps.txt'

# Create two files with filenames list of event-images and exposure-maps
#
create_obsfilelist $PWD 'object_observations.csv' $EVENTSFILE $EXMAPSFILE

# Sum the event-images
#
script_xselect_sum 'object' $EVENTSFILE 'events_sum.xcm' 'evts'

# Sum the exposure-maps
#
script_ximage_sum 'object' $EXMAPSFILE 'exmaps_sum.xco' 'expo'
```

### Run XImage scripts

Now it is time to run `events_sum.xco` and `exmaps_sum.xco` on `ximage` to have the final/stacked images.

For clarity, the content of the above `xco` files are *echo'd* below...

In [45]:
!cat 1ES/events_sum.xcm

xsel
read ev sw00030376079xpcw3po_cl.evt.gz
tmp_dir/
yes
read ev sw00030376060xpcw3po_cl.evt.gz
extract ev
read ev sw00030376080xpcw3po_cl.evt.gz
extract ev
read ev sw00030376059xpcw3po_cl.evt.gz
extract ev
read ev sw00030376087xpcw3po_cl.evt.gz
extract ev
read ev sw00030376072xpcw3po_cl.evt.gz
extract ev
read ev sw00030376062xpcw3po_cl.evt.gz
extract ev
read ev sw00030376086xpcw3po_cl.evt.gz
extract ev
read ev sw00030376063xpcw3po_cl.evt.gz
extract ev
read ev sw00030376073xpcw3po_cl.evt.gz
extract ev
read ev sw00030376076xpcw3po_cl.evt.gz
extract ev
read ev sw00030376067xpcw3po_cl.evt.gz
extract ev
read ev sw00030376071xpcw3po_cl.evt.gz
extract ev
read ev sw00030376033xpcw3po_cl.evt.gz
extract ev
read ev sw00030376032xpcw3po_cl.evt.gz
extract ev
read ev sw00030376083xpcw3po_cl.evt.gz
extract ev
read ev sw00030376069xpcw3po_cl.evt.gz
extract ev
read ev sw00030376061xpcw3po_cl.evt.gz
extract ev
read ev sw00030376089xpcw3po_cl.evt.gz
extract ev
read ev sw00030376068xpcw3po_cl.evt.gz
extr

In [46]:
!cat 1ES/exmaps_sum.xco

cpd  object_sum_expo.gif/gif
read/size=800  /input/example/1ES/swift/data/obs/2017_02/00030376079/xrt/products/sw00030376079xpc_ex.img.gz
read/size=800  /input/example/1ES/swift/data/obs/2016_03/00030376060/xrt/products/sw00030376060xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2017_02/00030376080/xrt/products/sw00030376080xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2016_02/00030376059/xrt/products/sw00030376059xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2017_04/00030376087/xrt/products/sw00030376087xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2016_05/00030376072/xrt/products/sw00030376072xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2016_03/00030376062/xrt/products/sw00030376062xpc_ex.img.gz
sum_image
save_image
read/size=800  /input/example/1ES/swift/data/obs/2017_04/00030376086/xrt/products/

Running the above scripts, will give us the iamges below.
```
xselect < events_sum.xcm
ximage < exmaps_sum.xco
```

### Event-images stack:

![Events](object_sum_evts.gif)

### Exposure-maps stack:

![Exposure maps](object_sum_expo.gif)

## Who's inside Stripe82?

In [59]:
from numpy import logical_or,logical_and
ra_idx = logical_or(table_master['RA'].between(0,60),table_master['RA'].between(330,360))
dec_idx = table_master['DEC'].between(-1.5,1.5)
table_stripe82 = table_master.loc[logical_and(ra_idx,dec_idx)]
table_stripe82

,NAME,ORIG_TARGET_I,TARGET_ID,RA,DEC,START_TIME,STOP_TIME,ORIG_OBS_SEGM,OBS_SEGMENT,ORIG_OBSID,OBSID,XRT_EXPOSURE,XRT_EXPO_PC,ARCHIVE_DATE,SOFTWARE_VERS,PROCESSING_DA,PROCESSING_VE
0,NON-SCIENCE DATA,60142,60142,0.640524,0.278400,04/12/08 05:50:01,04/12/08 14:29:44,0,0,60142000,60142000,6665.174,2201.420,04/12/15 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/12 00:00:14,3.7.6
1,NON-SCIENCE DATA,60140,60140,0.640524,0.278400,04/12/07 23:59:16,04/12/08 06:37:25,0,0,60140000,60140000,10642.695,8751.410,04/12/14 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/14 00:00:14,3.7.6
2,NON-SCIENCE DATA,60150,60150,0.640524,0.278400,04/12/09 05:48:31,04/12/09 13:22:42,0,0,60150000,60150000,7307.151,2931.921,04/12/16 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/14 00:00:14,3.7.6
3,NON-SCIENCE DATA,60148,60148,0.640524,0.278400,04/12/08 23:59:14,04/12/09 06:44:02,0,0,60148000,60148000,12749.664,9399.110,04/12/15 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/14 00:00:14,3.7.6
4,NON-SCIENCE DATA,60156,60156,0.640524,0.278400,04/12/09 23:59:14,04/12/10 06:43:19,0,0,60156000,60156000,12687.397,9569.049,04/12/16 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/12 00:00:14,3.7.6
5,NON-SCIENCE DATA,60146,60146,0.640524,0.278400,04/12/08 17:17:18,04/12/09 00:39:19,0,0,60146000,60146000,3892.175,2716.944,04/12/15 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/13 00:00:14,3.7.6
6,NON-SCIENCE DATA,60138,60138,0.640524,0.278400,04/12/07 17:59:17,04/12/08 00:45:14,0,0,60138000,60138000,4074.440,2704.442,04/12/14 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/12 00:00:14,3.7.6
7,NON-SCIENCE DATA,60152,60152,0.640524,0.278400,04/12/09 11:42:05,04/12/10 00:05:14,0,0,60152000,60152000,6332.973,1080.260,04/12/16 00:00:13,HEA_20JULY2006_V6.1_SWIFT_REL2.5A(BLD19)_22SEP2006,06/11/13 00:00:14,3.7.6
49716,1RXSJ005350.4-013023,83050,83050,13.461141,-1.498985,16/05/19 17:26:12,16/05/19 18:19:41,1,1,83050001,83050001,451.305,451.305,16/05/30 00:00:14,HEA_27JUL2015_V6.17_SWIFT_REL4.5(BLD34)_27JUL2015_,16/05/29 00:00:14,3.17.03
49731,KEPLER_REG7_PT20,85729,85729,18.431645,-1.476339,15/09/11 16:59:11,15/09/11 17:51:25,1,1,85729001,85729001,526.525,526.525,15/09/22 00:00:14,HEA_21DEC2012_V6.13_SWIFT_REL4.0(BLD29)_14DEC2012_,15/09/21 00:00:14,3.16.09


In [62]:
objects_s82 = table_stripe82.NAME.unique()
objects_s82.sort()
objects_s82

array(['1FGLJ2223.3+0103', '1RXSJ003238.2-010056', '1RXSJ004135.7-012143',
       '1RXSJ004547.2-005034', '1RXSJ004941.8-011657',
       '1RXSJ005350.4-013023', '1RXSJ010227.1-003911',
       '1RXSJ011202.7-004355', '1RXSJ013051.8+012712',
       '1RXSJ013353.5+011343', '1RXSJ014942.9+001504',
       '1RXSJ015105.7-003427', '1RXSJ020146.5+011717',
       '1RXSJ020220.5-010609', '1RXSJ020221.0-010715',
       '1RXSJ020622.7-011754', '1RXS_J000709.8+005328',
       '1RXS_J001821.7+011859', '1RXS_J002147.4+000036',
       '1RXS_J003846.3+003430', '1RXS_J005441.9+000056',
       '1RXS_J010243.0+010805', '1RXS_J010939.6+005943',
       '1RXS_J011704.2+000025', '1RXS_J012202.5+002023',
       '1RXS_J013137.7+003251', '1RXS_J013353.5+011343',
       '1RXS_J014239.9+000514', '1RXS_J014942.9+001504',
       '1RXS_J015910.0+010518', '1RXS_J020002.0+012123',
       '1RXS_J020106.3+003400', '1RXS_J020151.6+012902',
       '1RXS_J020432.4+012208', '1RXS_J021359.6+004245',
       '1RXS_J024241.2+005

### Resolv objects...

In [79]:
def resolve_name(name):
    '''
    Return ICRS position for given object 'name'
    Input:
     - name : str
        Object designation/name
    Output:
     - position : (float,float)
        Tuple with (RA,Dec) coordinates in degree
    '''
    from astropy.coordinates import get_icrs_coordinates as get_coords
    try:
        icrs = get_coords(name)
        pos = (icrs.ra.value,icrs.dec.value)
    except:
        pos = None
    return pos


import re
valid_objects = []
for obj in objects_s82:
    mat = re.search('J\d',obj)
    if not (mat and mat.start()>0):
        continue
    start = mat.start()
    oprj = re.sub('_','',obj[:start])
    opos = obj[start:]
    obj = '{} {}'.format(oprj,opos)
    print("Resolving object '{}' position".format(obj))
    pos = resolve_name(obj)
    if pos != None:
        print('...position: {!s}'.format(pos))
        valid_objects.append(obj)
    else:
        print('...position NOT resolved')


Resolving object '1FGL J2223.3+0103' position
...position: (335.829, 1.064)
Resolving object '1RXS J003238.2-010056' position
...position: (8.1591667, -1.0155556)
Resolving object '1RXS J004135.7-012143' position
...position: (10.39875, -1.3619444)
Resolving object '1RXS J004547.2-005034' position
...position: (11.4647337, -0.8601794)
Resolving object '1RXS J004941.8-011657' position
...position: (12.4241667, -1.2825)
Resolving object '1RXS J005350.4-013023' position
...position: (13.4580833, -1.5083889)
Resolving object '1RXS J010227.1-003911' position
...position: (15.6096601, -0.6512789)
Resolving object '1RXS J011202.7-004355' position
...position: (18.0168926, -0.7310771)
Resolving object '1RXS J013051.8+012712' position
...position: (22.71875, 1.4525)
Resolving object '1RXS J013353.5+011343' position
...position: (23.4694129, 1.2292108)
Resolving object '1RXS J014942.9+001504' position
...position: (27.4271351, 0.250477)
Resolving object '1RXS J015105.7-003427' position
...positi

...position NOT resolved
Resolving object 'SDSS J2234+0041' position
...position NOT resolved
Resolving object 'SDSS J2238+0108' position
...position NOT resolved
Resolving object 'SDSS J224028.85-010649.8' position
...position: (340.1202284, -1.1138646)
Resolving object 'SDSS J225409.59+005629.1' position
...position: (343.5399788, 0.9414348)
Resolving object 'SDSS J225913.92-00425' position
...position NOT resolved
Resolving object 'SDSS J230155.54-010649.0' position
...position: (345.4814575, -1.1136231)
Resolving object 'SDSS J2303+0106' position
...position NOT resolved
Resolving object 'SDSS J230609.22+000844.2' position
...position: (346.5384202, 0.145617)
Resolving object 'SDSS J231731.4-001605' position
...position: (349.380694, -0.268036)
Resolving object 'SDSS J232259.99-005359.3' position
...position: (350.7499631, -0.8998053)
Resolving object 'SDSS J232712.98-0036' position
...position NOT resolved
Resolving object 'SDSS J233604.04+000447.1' position
...position: (354.0168

In [80]:
valid_objects

['1FGL J2223.3+0103',
 '1RXS J003238.2-010056',
 '1RXS J004135.7-012143',
 '1RXS J004547.2-005034',
 '1RXS J004941.8-011657',
 '1RXS J005350.4-013023',
 '1RXS J010227.1-003911',
 '1RXS J011202.7-004355',
 '1RXS J013051.8+012712',
 '1RXS J013353.5+011343',
 '1RXS J014942.9+001504',
 '1RXS J015105.7-003427',
 '1RXS J020146.5+011717',
 '1RXS J020220.5-010609',
 '1RXS J020221.0-010715',
 '1RXS J020622.7-011754',
 '1RXS J000709.8+005328',
 '1RXS J001821.7+011859',
 '1RXS J002147.4+000036',
 '1RXS J003846.3+003430',
 '1RXS J005441.9+000056',
 '1RXS J010243.0+010805',
 '1RXS J010939.6+005943',
 '1RXS J011704.2+000025',
 '1RXS J012202.5+002023',
 '1RXS J013137.7+003251',
 '1RXS J013353.5+011343',
 '1RXS J014239.9+000514',
 '1RXS J014942.9+001504',
 '1RXS J015910.0+010518',
 '1RXS J020002.0+012123',
 '1RXS J020106.3+003400',
 '1RXS J020151.6+012902',
 '1RXS J020432.4+012208',
 '1RXS J021359.6+004245',
 '1RXS J024241.2+005715',
 '1RXS J030145.1+011538',
 '1RXS J032730.1-005950',
 '2FGL J0158.4+0